# Active work in LIFE Target Database

### distinct values bug

In [3]:
import numpy as np
test_table=database_tables['star_basic']['sptype_string','teff_st_value','mass_st_value','mass_st_source_idref','radius_st_value','radius_st_source_idref']
print(test_table)

sptype_string teff_st_value ...   radius_st_value   radius_st_source_idref
                    K       ...        solRad                             
------------- ------------- ... ------------------- ----------------------
          F6V        6350.0 ...  1.6382520198822021                    876
                      1e+20 ...               1e+20                 999999
       M5.5Ve        2930.0 ...               0.156                    467
           F8        6062.0 ...  1.3733317852020264                    876
         F7Vs        6280.0 ...               1.324                    467
        F5.5V         1e+20 ...               1e+20                 999999
         F8IV        6115.0 ...  2.0629305839538574                    876
           K4        4600.0 ...               0.713                    467
          F5V        6422.0 ...  1.4377461671829224                    876
          ...           ... ...                 ...                    ...
       M0V(e)        3850

#### Issue of same mass e.g. about 0.4, many different radii e.g. 0.2-0.6 all on one line

In [6]:
samemass=test_table['sptype_string','mass_st_source_idref','radius_st_value','radius_st_source_idref'][np.where(test_table['radius_st_value']<0.5)]
mass_zoom_in=samemass[np.where(samemass['radius_st_value']>0.3)]
print(mass_zoom_in)

sptype_string mass_st_source_idref   radius_st_value   radius_st_source_idref
                                          solRad                             
------------- -------------------- ------------------- ----------------------
                            999999  0.3259684443473816                    876
         M2.5                  467               0.421                    467
           M2                  467               0.446                    467
           M3                  467 0.33001458644866943                    876
                            999999  0.3244355320930481                    876
          M3:                  467  0.4157400131225586                    876
        M3.5V                  467 0.37713417410850525                    876
        M3.0V                  467               0.361                    467
        M2.5V                  467 0.45737722516059875                    876
          ...                  ...                 ...          

In [32]:
rad37=test_table[np.where(test_table['mass_st_value']==0.37)]
print(rad37[np.where(rad37['radius_st_value']!=0.361)])
#alright, now get ref for them, maybe they are gaia things. if so, then put ref in catalog creation. if not find bug
#all from source 879 -> gaia
#those that are from life should be 467

#check how quality is assigned in gaia radius (as compared to mass and temparature)

sptype_string teff_st_value ...   radius_st_value   radius_st_source_idref
                    K       ...        solRad                             
------------- ------------- ... ------------------- ----------------------
           M3        3430.0 ...  0.5470667481422424                    876
           M3        3430.0 ... 0.33001458644866943                    876
          M3:        3430.0 ...  0.4157400131225586                    876
         M3Ve        3430.0 ...  0.3676148056983948                    876
       M3.0Ve        3430.0 ...  0.5073124170303345                    876
           M3        3430.0 ... 0.15253719687461853                    876
           M3        3430.0 ...  0.6825898289680481                    876
        M3.0V        3430.0 ...  0.5819295644760132                    876
          M3e        3430.0 ...  0.4614301025867462                    876
          ...           ... ...                 ...                    ...
           M3        3430

In [ ]:
#check out radius quality
#just assigning B, model is C. I could get upper and lower limits. Might be able to create a quality out of those. Don't think that is useful right now.
#don't think have mass measurements for those stars.

In [9]:
mass_table=database_tables['mes_mass_st']
gaia_mass_table=mass_table[np.where(mass_table['mass_st_source_idref']==876)]
print(gaia_mass_table)
#about 1 000 of the 20 000 stars have a gaia mass measurement

  main_id      mass_st_value    mass_st_qual ... mass_st_err object_idref
                  solMass                    ...                         
------------ ------------------ ------------ ... ----------- ------------
   *   1 Hya 1.2794208526611328            B ...       1e+20            1
   *   3 Cyg 1.1393938064575195            B ...       1e+20            4
   *   5 Ser 1.2650383710861206            B ...       1e+20            7
   *   6 And 1.2294059991836548            B ...       1e+20            9
   *   6 Cet 1.2543262243270874            B ...       1e+20           10
   *   7 And 1.5735499858856201            B ...       1e+20           11
   *   8 Dra 1.4817147254943848            B ...       1e+20           12
   *   9 Cet 0.9415072202682495            B ...       1e+20           17
   *  10 CVn 1.0280797481536865            B ...       1e+20           19
         ...                ...          ... ...         ...          ...
V* V2213 Oph 1.0495580434799194       

In [10]:
mass_table=database_tables['mes_radius_st']
gaia_mass_table=mass_table[np.where(mass_table['radius_st_source_idref']==876)]
print(gaia_mass_table)

         main_id            radius_st_value   ... radius_st_err object_idref
                                 solRad       ...                           
------------------------- ------------------- ... ------------- ------------
                *   1 Hya  1.6382520198822021 ...         1e+20            1
                *   3 Cyg  1.3733317852020264 ...         1e+20            4
                *   5 Ser  2.0629305839538574 ...         1e+20            7
                *   6 And  1.4377461671829224 ...         1e+20            9
                *   6 Cet  1.6369404792785645 ...         1e+20           10
                *   7 And   1.846876859664917 ...         1e+20           11
                *   8 Dra  1.5851472616195679 ...         1e+20           12
                *   9 Cet  1.0720789432525635 ...         1e+20           17
                *  10 CVn   1.019992709159851 ...         1e+20           19
                      ...                 ... ...           ...          ...

I found the issue for the big spread in lower radii for only discrete values of masses. Not sure what to do about it though.So the issue is, that I prefer gaia radius and mass over model values. I don't have a concrete reason for it, just assuming that gaia is better.
Gaia gives only very few mass measurements (about 1 000 of 20 000 stars (systems included) have them). So in the catalog I mostly get model values for masses and those are discrete. On the other hand I do get about 4'000 radius measurements from gaia.So what we need to figure out: Is this okay for us or do we want to change it?
In the later case I would need to find a way to assign a quality to individual gaia radius measurements. Maybe gaia provides something like that that I just did not find, maybe I can create one from the upper and lower limits that gaia does provide but I currently not include in the database and maybe somebody has since then created that already.
For context: for gaia temperatures I do assign individual qualities for those from gsp-spec (because they offer some kind of quality flag) but not for gsp-phot (where I found an external publication which would do so but run into storage issues for large data). The first one sometimes gets prefered over the model, the later never.

#### Issue of same temperature, many different radii e.g. about 3300 and 0.2 to 0.8

In [ ]:
zoom_in=test_table[np.where(test_table['teff_st_value']<3500)]
zoom_in=zoom_in[np.where(zoom_in['teff_st_value']>3400)]
print(zoom_in)

In [ ]:
print(test_table[np.where(test_table['teff_st_value']==3470)]
)

### Missing stars in new starcat4 as compared to previus version

Lia sais 10% missing -> See catalog directory, StarCat_testbench notebook

## starcat 4 stars that would be good to be in there

Jonah: golden targets

Felix: a lot fewer k and g stars in version 4 than version 3 catalog

In [1]:
from utils.analysis.object_analysis import starcat_versions_lost_stars
lost_g_stars = starcat_versions_lost_stars()
print(lost_g_stars)



 main_id  
----------
 *  20 LMi
 HD  82443
 HD 220140
 *  11 LMi
 *  15 LMi
 *  15 Sge
 *  18 Sco
 *  39 Tau
 *  47 UMa
 *  61 UMa
       ...
 HD  69830
 HD 135204
 HD 157347
 HD 165185
 HD 172051
 HD 190771
 HD 207129
 HD 211415
HD 222834A
 V* HN Peg
Length = 36 rows


In [ ]:

# quite a number fail because of wds companions not having all the parameters we need -> would be solved by using dedicated multiplicity catalog for those. though some of those should have sptype in simbad, why do I not get that?
# - simbad object is not included... why not? create unit test for this to figure out
# some have issue that system name and primary name are the same so I get systems without children

In [3]:
# 1. have a first look at it

from utils.analysis.object_analysis import detail_info_object

starname='WDS J10114+4927B'#'*  11 LMi'
detail_info_object(database_tables,starname)
#wait, eta cas A is seen as system but does have stellar parameters...
#why is not having a parent a reason for exclusion?


Identifier 
       id            main_id      id_source_idref object_idref
---------------- ---------------- --------------- ------------
WDS J10114+4927B WDS J10114+4927B             197        18143

Object type
type
----
  st

 Basic stellar data
    main_id      class_lum class_temp class_temp_nr
---------------- --------- ---------- -------------
WDS J10114+4927B         ?          ?             ?

Parents
  parent_main_id   child_main_id   ... child_object_idref parent_object_idref
                                   ...                                       
----------------- ---------------- ... ------------------ -------------------
WDS J10114+4927AB WDS J10114+4927B ...              18143               18141

Siblings
  parent_main_id   child_main_id   ... child_object_idref parent_object_idref
                                   ...                                       
----------------- ---------------- ... ------------------ -------------------
WDS J10114+4927AB        HD 

In [4]:
# 2. run the code specifically made to find out reason

import importlib
#refactured code: -> seems to be working
#now trying to print out and / or store as list exactly objects that lead to exclusion e.g. children
#hm I guess I would like to have a reference to which catalog
#   I used to assicn the type of the object (gaia, wds, simbad)
import utils.analysis.catalog_comparison as cc
importlib.reload(cc)

# list given by jonah
l_golden_targets=['* alf Cen A','* alf Cen B','* alf Cen C','NAME Teegarden\'s Star','* tau Cet','* eps Eri','* eps Ind','HD  88230','* omi02 Eri','* sig Dra','HD 131977','* eta Cas A','* eta Cas B','TRAPPIST-1']
# list given by felix lost_g_stars

cc.detail_criteria(database_tables,list(lost_g_stars))

tbd: file starcat4 analysis where this code is used to give specific reason why a given object was not included into the cat4
system object but with found child object *  20 LMi
child_main_id
-------------
   HD  86728b
system object but with found child object HD  82443
      child_main_id      
-------------------------
sdb-v2-093243.76+265918.7
star's parents
  parent_main_id  child_main_id ... child_object_idref parent_object_idref
                                ...                                       
----------------- ------------- ... ------------------ -------------------
WDS J23194+7900AB     HD 220140 ...               9461               21344
WDS J23194+7900AC     HD 220140 ...               9461               21345
WDS J23194+7900AD     HD 220140 ...               9461               21346
WDS J23194+7900AE     HD 220140 ...               9461               21347
star's parents
  parent_main_id  child_main_id ... child_object_idref parent_object_idref
                    

### issue non physical multiples

### issue out of distance cut h_link


In [ ]:
distance_cut_in_pc = 30.
plx_in_mas_cut = 1000. / distance_cut_in_pc
print(plx_in_mas_cut)
#making cut a bit bigger for correct treatment of objects on boundary
plx_cut = plx_in_mas_cut - plx_in_mas_cut / 10.
print(plx_cut)
#something is wrong or is it just that 30 pc gives a strange result?, yes 30 pc is mathematically strange

In [ ]:
 # hm WDS J10114+4927B seems to have a way too small plx to be in our sample. so is just visual binary from wds
    # how do I incorporate this knowledge into my db? if I have this object in my db I should include its plx too
    # if I kick it out I should indicate that sibling is actually single star
    # ... maybe in life provider take the wds objects, fetch main_id, get simbad stuff for them,
    # run specific multiplicity catalog over them, adapt binary & h-link stuff
    # sounds more like something that should be done in wds provider.
    # right when creating the helpertable
    # like as soon as I have an id I should also query the plx
    # make a list of those whose plx are given but outside our cutoff
    # make a list of their associated objects
    # or right after running wds I take all objects of type sy and run a new provider with better
    # next idea:
    # at end of life provider (which should run last) get all possible plx and clean out objects that are too far out. give hosts and siblings special treatment e.g. a flag



In [ ]:
from provider.simbad import *
from astropy.table import Table


def test_wds_nonphysical_multiples():
    # data
    distance_cut_in_pc = 30.
    test_objects = ['WDS J10114+4927B']
    cat = Table(data=[['WDS J10114+4927B'],['HD 233714']],
                names=['id','main_id'],
                dtype=[object,object])
    # hm WDS J10114+4927B seems to have a way too small plx to be in our sample. so is just visual binary from wds
    # how do I incorporate this knowledge into my db? if I have this object in my db I should include its plx too
    # if I kick it out I should indicate that sibling is actually single star
    # ... maybe in life provider take the wds objects, fetch main_id, get simbad stuff for them,
    # run specific multiplicity catalog over them, adapt binary & h-link stuff

    # get main id of test objects
    fetch_main_id(cat, id_creator = IdentifierCreator(name='main_id', colname='id'))

    # function (problem, this is more of an integration test but the function
    #  is written in a way to enable easy exploration testing)
    sim_helptab, sim = create_simbad_helpertable(distance_cut_in_pc, test_objects)

    # assert

In [ ]:



#'* 212 Pup A' kicked out in non trivial binaries in starcat but in this code as trivial binary with companion not matching sptype
#['* zet01 Ret', '*  50 Per', 'HD  50281', 'HD  65907', '* eps Ind', 'V* V2215 Oph'] other way round

# database creation

In [1]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

provider_tables_dict, database_tables= ltd.partial_create(30,create=['life'])

#provider_tables_dict, database_tables= ltd.create_life_td(30)

#issue: does not wokr with loading as then id_source_idref already exists -> put in a workaround with a warning. though now did not happen...

#issue: create not working right now because exomercat server not available -> put in a workaround
#issue: main_id missing in star_basic -> suddenly working??

#vizier not working? -> do tests first -> issue on topcat too -> solved

Building life_td data with distance cut of 30.0 pc
Loading sim data
Loading sdb data
Loading wds data
Loading exo data
Trying to create LIFE tables from 2025-12-03...
sim_objects  main_id  ... type
--------- ... ----
HD  61606 ...   sy
 main_id      coo_gal_l          coo_gal_b     ... class_lum class_ref
                 deg                deg        ...                    
--------- ------------------ ----------------- ... --------- ---------
HD  61606 221.75957890075676 9.099562783385588 ...         V      LIFE
 main_id  ... class_ref
          ...          
--------- ... ---------
HD  61606 ...      LIFE
 main_id   mod_M 
          solMass
--------- -------
HD  61606    0.78
Loading gaia data
Building sources table ...
Building objects table ...


Building provider table ...
Building ident table ...


Building best_h_link table ...
Building star_basic table ...
Building planet_basic table ...
Building disk_basic table ...
Building mes_mass_pl table ...
Building mes_teff_st table ...
Building mes_radius_st table ...
Building mes_mass_st table ...
Building mes_binary table ...
Building mes_sep_ang table ...
Building h_link table ...


Unifying null values...
Saving data...


# load and analyze database

In [1]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

provider_tables_dict, database_tables= ltd.load_life_td()

#ah when it showed me an empty area in wds.py it actually didn't load it correctly. when I then cleaned it out it introduced an error. need to uncommit the minor fixes commit and then unstack that file

Loading life_td generated data
Loading sim data
Loading sdb data
Loading wds data
Loading exo data
Loading life data
Loading gaia data


In [2]:
import numpy as np
#catalog=provider_tables_dict['life']['star_basic']
catalog=database_tables['star_basic']
print(catalog.colnames)
#nomass=catalog['type'][np.where(catalog['main_id']=='HD  61606')]
nomass=catalog['main_id', 'mass_st_value','sptype_string',"class_lum","class_temp","class_temp_nr"][np.where(catalog['main_id']=='HD  61606')]
print(nomass)
#has no mass assigned but test is working. lets try after having db created anew
#hm HD  61606 object not in life mes_mass_st. but is in star_basic and simbad. in life I only take stars, is it a system? in simbad it is sy, in total it is st. so there lies the issue.

['main_id', 'coo_ra', 'coo_dec', 'coo_err_angle', 'coo_err_maj', 'coo_err_min', 'coo_qual', 'coo_ref', 'mag_i_value', 'mag_i_ref', 'mag_j_value', 'mag_j_ref', 'mag_k_value', 'mag_k_ref', 'sptype_string', 'sptype_qual', 'sptype_ref', 'plx_value', 'plx_err', 'plx_qual', 'plx_ref', 'coo_source_idref', 'plx_source_idref', 'mag_i_source_idref', 'mag_j_source_idref', 'mag_k_source_idref', 'sptype_source_idref', 'coo_gal_l', 'coo_gal_b', 'coo_gal_err_angle', 'coo_gal_err_maj', 'coo_gal_err_min', 'coo_gal_qual', 'coo_gal_ref', 'dist_st_value', 'dist_st_ref', 'class_temp', 'class_temp_nr', 'class_lum', 'class_ref', 'dist_st_source_idref', 'coo_gal_source_idref', 'class_source_idref', 'mag_i_err', 'mag_i_qual', 'mag_j_err', 'mag_j_qual', 'mag_k_err', 'mag_k_qual', 'dist_st_err', 'dist_st_qual', 'sptype_err', 'object_idref', 'teff_st_value', 'teff_st_err', 'teff_st_qual', 'teff_st_source_idref', 'radius_st_value', 'radius_st_err', 'radius_st_qual', 'radius_st_source_idref', 'mass_st_value', 'mass

## gaia quality flags assignment

Bug: Finding issue that causes M0-M5 stars to have too high temperatures (4250 instead of 3000).

Status: I think I partially solved it by coding part of gsp-spec to include quality flags. doesn't seem to work correctly yet and phot quality not taken into account yet

In [28]:
import numpy as np
#9=mesteff
#5=starbasic
#print(database_tables[5].colnames)
cols=['main_id','sptype_string','teff_st_qual','teff_st_value','teff_st_source_idref']
mstars=database_tables['star_basic'][cols][np.where(database_tables['star_basic']['class_temp']=='M')]
mstars_tempgiven=mstars[np.where(mstars['teff_st_value']!=1e+20)]
interesting_sample=mstars_tempgiven[np.where(mstars_tempgiven['teff_st_value']>4000)]
print(interesting_sample)
# shows that I do have a couple of M stars with unlikely high temperatures

  main_id   sptype_string teff_st_qual   teff_st_value   teff_st_source_idref
                                               K                             
----------- ------------- ------------ ----------------- --------------------
BD-13  1434           M0V            B   4269.0712890625                  878
BD-22  6219           M0V            C            4157.0                  879
CD-26 11751            M1            C            4008.0                  879
CD-34  1169             M            D            4250.0                  879
CD-36  9107            M0            B 4095.856689453125                  878
CD-49 14193           M0V            C            4059.0                  879
  HD 166348           M0V            B    4235.400390625                  878
  HD 288595            M0            C            4096.0                  879
  HD 330856            M1            C            4105.0                  879
  HD 356314            M0            B   4104.5791015625        

In [29]:
#source_id = interesting_sample['teff_st_source_idref'][0] # GAIA GSP-Phot
source_id = interesting_sample['teff_st_source_idref'][1] # GAIA GSP-Spec
print(database_tables['sources'][np.where(database_tables['sources']['source_id']==source_id)])
# shows that source is gaia

            ref              provider_name source_id
---------------------------- ------------- ---------
2022arXiv220800211G GSP-Spec          Gaia       879


In [30]:
import numpy as np
# finding out, if it is in gaia provider right or if there is an issue when building db together

cols=['teff_st_qual','teff_st_value','teff_st_source_idref']

print(database_tables['mes_teff_st'][cols][np.where(database_tables['mes_teff_st']['main_id']=='HD 166348')])


teff_st_qual teff_st_value  teff_st_source_idref
                   K                            
------------ -------------- --------------------
           C         3850.0                  468
           B         4108.0                  879
           B 4235.400390625                  878


In [31]:
from sdata import empty_dict
from provider.utils import fetch_main_id, IdentifierCreator, create_provider_table, query
from pyvo.dal import TAPService

adql_query="""
    SELECT top 10 s.source_id ,p.mass_flame, p.radius_flame,
        p.teff_gspphot, p.teff_gspspec, m.nss_solution_type, p.age_flame, p.flags_gspspec
    FROM gaiadr3.gaia_source as s
        JOIN gaiadr3.astrophysical_parameters as p ON s.source_id=p.source_id
            LEFT JOIN gaiadr3.nss_two_body_orbit as m ON s.source_id=m.source_id
            WHERE s.source_id = 4848140361962951552"""
gaia = empty_dict.copy()
gaia['provider'] = create_provider_table('Gaia',
                                  "https://gea.esac.esa.int/tap-server/tap",
                                  '2016A&A...595A...1G')
try:
    gaia_helptab=query(gaia['provider']['provider_url'][0],adql_query)

except:
    #because of bug in gaia server where async not working currently
    service = TAPService(gaia['provider']['provider_url'][0])
    result=service.run_sync(adql_query.format(**locals()), maxrec=160000)
    gaia_helptab=result.to_table()


Trying to create Gaia tables from 2025-06-26...


In [32]:
#so lets find out if in gaia archive this is also the case
# find gaia id
#Gaia DR3 4848140361962951552

#on archive only gspphot showable and it is empty

In [33]:
print(gaia_helptab['source_id','teff_gspspec'])

     source_id      teff_gspspec
                         K      
------------------- ------------
4848140361962951552       4250.0


In [34]:
#so yes that is how it is in gaia. now find out if this parameter is reliable and if not lower its quality

In [35]:
# so I did add a quality assignment for spec, those should be lower quality now sometimes
# is the case in star_basic of dtabase_tables
print(interesting_sample[np.where(interesting_sample['teff_st_source_idref']==source_id)])

  main_id   sptype_string teff_st_qual teff_st_value teff_st_source_idref
                                             K                           
----------- ------------- ------------ ------------- --------------------
BD-22  6219           M0V            C        4157.0                  879
CD-26 11751            M1            C        4008.0                  879
CD-34  1169             M            D        4250.0                  879
CD-49 14193           M0V            C        4059.0                  879
  HD 288595            M0            C        4096.0                  879
  HD 330856            M1            C        4105.0                  879
V* V430 Pup            M0            C        4075.0                  879


In [36]:
# how about mes_teff_st table?

print(database_tables['mes_teff_st'][cols][np.where(database_tables['mes_teff_st']['main_id']=='HD 166348')])
# does not seem to be lower quality,
# either its quality flag is especially good or it did not get updated in the mes table


teff_st_qual teff_st_value  teff_st_source_idref
                   K                            
------------ -------------- --------------------
           C         3850.0                  468
           B         4108.0                  879
           B 4235.400390625                  878


In [37]:
# check quality flag of this object gsp-spec
# do I have the quality flag still in mes table? no
print(database_tables['mes_teff_st'].colnames)

['main_id', 'teff_st_value', 'teff_st_qual', 'teff_st_ref', 'teff_st_source_idref', 'teff_st_err', 'object_idref']


In [38]:
# take it directly from specific gaia query
print(gaia_helptab['flags_gspspec'])

              flags_gspspec              
-----------------------------------------
00100131000019999999999999999999999999999


In [39]:
flag=gaia_helptab['flags_gspspec'][0]
summed=0
interval = 41 * 9 / 5.
for i in flag:
    summed += int(i)
print(summed)

if summed in range(0, int(interval) + 1):
    print('A')
elif summed in range(int(interval) + 1, int(interval * 2) + 1):
    print('B')
elif summed in range(int(interval * 2) + 1, int(interval * 3) + 1):
    print('C')
elif summed in range(int(interval * 3) + 1, int(interval * 4) + 1):
    print('D')
elif summed in range(int(interval * 4) + 1, int(interval * 5) + 1):
    print('E')

# so it looks like should be low quality and value in mes_teff_st is wrong

259
D


In [48]:
# lets test if in the gaia provider is correct in mes_teff_st
name='HD 166348'
print(provider_tables_dict['gaia']['mes_teff_st'][np.where(provider_tables_dict['gaia']['mes_teff_st']['main_id']==name)])
# shows is there quality of B too instead of D

 main_id  teff_st_value teff_st_qual         teff_st_ref         
                K                                                
--------- ------------- ------------ ----------------------------
HD 166348     4235.4004            B 2022arXiv220800211G GSP-Phot
HD 166348        4108.0            B 2022arXiv220800211G GSP-Spec


In [41]:
# go to gaia file and find line where should get lowered. then create some unit tests for it -> line 174
# the assign quality unit test works with this data too so now find line where it gets written into mes_teff
# maybe 150 function creating teff_st -> created test for it
# now most likely integration test for whole gaia needed

In [42]:
# how about phot?

In [43]:
import utils.analysis as lta

lta.sanity_tests(database_tables,5)

AttributeError: module 'utils.analysis' has no attribute 'sanity_tests'